# Importing Necessary Libraries

In [1]:
!pip install chromadb
!pip install sentence_transformers

In [2]:
import json
import pandas as pd
import numpy as np
import chromadb
from sentence_transformers import SentenceTransformer

# Importing the Dataset

In [3]:
with open('References.json', 'r', encoding='utf-8') as f:
    ref = json.load(f)

In [4]:
ref

[{'id': 'story_001',
  'title': "The Brahmadaitya's Bargain",
  'region': 'India',
  'culture': 'Bengali',
  'creatures': ['Brahmadaitya', 'spirit'],
  'type': 'Folktale',
  'themes': ['ghost', 'wisdom', 'village'],
  'text': 'A benevolent Brahmadaitya, bound to a banyan tree, offers guidance to a frightened traveler who lost the road at dusk. In exchange for companionship, the spirit shares forgotten histories of the village and helps the man expose a corrupt landlord.'},
 {'id': 'story_002',
  'title': 'Kitsune at the Riverbank',
  'region': 'Japan',
  'culture': 'Japanese',
  'creatures': ['Kitsune'],
  'type': 'Folktale',
  'themes': ['shapeshifter', 'trickster', 'love'],
  'text': 'A fox spirit disguised as a musician helps a lonely farmer recover from grief. When her identity is revealed by moonlight, she vanishes in a swirl of leaves but leaves him a flute that summons foxfire whenever he feels lost.'},
 {'id': 'story_003',
  'title': 'Song of the Banshee',
  'region': 'Ireland'

In [5]:
model = SentenceTransformer("all-MiniLm-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
client = chromadb.PersistentClient(path="./reference_db")

In [7]:
reference = client.get_or_create_collection(
    name="reference",
    metadata={"hnsw:space": "cosine"}
)

In [8]:
ids = []
docs = []
metas = []

In [9]:
for item in ref:

  txt = item['text']
  meta = {
      'id': item['id'],
      'title': item['title'],
      'region': item['region'],
      'culture': item['culture'],
      'creatures': ', '.join(item['creatures']),
      'themes': ', '.join(item['themes']),
      'vector_creatures': ' '.join(item['creatures']).lower(),
      'vector_region': f"{item['region']} {item['culture']} folklore".lower(),
      'vector_themes': ' '.join(item['themes']).lower()
  }

  ids.append(item['id'])
  docs.append(txt)
  metas.append(meta)

In [10]:
reference.add(
    documents=docs,
    metadatas=metas,
    ids=ids
)